In [2]:
import dxcam
from src.utils.windowtools import fuzzy_window_search, calculate_aspect_ratio, check_aspect_ratio_validity
import matplotlib.pyplot as plt
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import numpy as np
import cv2

camera = dxcam.create()


In [5]:
coords = fuzzy_window_search("asphalt")

aspect_ratio = calculate_aspect_ratio(coords)

check_aspect_ratio_validity(aspect_ratio)

[(1075, 148, 1746, 731)]
1
The aspect ratio is reasonable.


In [ ]:
# Grab a frame from the camera
window = camera.grab(region=coords)

# Display the frame
plt.imshow(window)
plt.axis('off')  # Hide the axis
plt.show()

NameError: name 'camera' is not defined

In [41]:
# First crop to top-right region
height, width, _ = window.shape
top_right_region = window[0:int(height*0.28), int(width*0.6):width]

# Target BGR values - easy to adjust
target_B = 228
target_G = 0
target_R = 0
tolerance = 5

# Create mask looking for the target color with tolerance
mask = np.zeros(top_right_region.shape[:2], dtype=np.uint8)
mask[
    (top_right_region[:,:,0] >= max(0, target_B - tolerance)) & 
    (top_right_region[:,:,0] <= min(255, target_B + tolerance)) &  # Blue channel
    (top_right_region[:,:,1] >= max(0, target_G - tolerance)) & 
    (top_right_region[:,:,1] <= min(255, target_G + tolerance)) &  # Green channel
    (top_right_region[:,:,2] >= max(0, target_R - tolerance)) & 
    (top_right_region[:,:,2] <= min(255, target_R + tolerance))  # Red channel
] = 255

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if contours:
    # Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Extract the region within the box
    box_region = top_right_region[y:y+h, x:x+w]
    
    # Convert to grayscale and extract text
    gray_region = cv2.cvtColor(box_region, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray_region, config='--psm 6')
    
    # Extract numbers while preserving time format
    numbers = ''.join(filter(str.isdigit, text))
    numbers = numbers[-7:] if len(numbers) >= 7 else numbers
    print("Extracted time:", numbers)
    #print(f"Number of contours found: {len(contours)}")
else:
    print("No contours found")

Extracted time: 0003464
